In [1]:
import pandas as pd
import os
from ast import literal_eval
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from typing import List
import numpy as np
import pandas as pd

import torch
#from sentence_transformers import SentenceTransformer, util

import re
import random

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

from nltk.stem import PorterStemmer  
porter_stemmer = PorterStemmer()

stop_words = set(stopwords.words())

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx
import community.community_louvain as community
from ast import literal_eval
import re
from collections import defaultdict, Counter

from utils import *

[nltk_data] Downloading package stopwords to /home/selim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/selim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/selim/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /home/selim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
def flatten(t):
    return [item for sublist in t for item in sublist]

DATA_PATH = os.path.join(
    '..', '..', '..', '..', "data", "frameworks_data", 'data_v0.7.1'
)

full_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset_with_translations.csv'))

kept_cols = ['sectors', 'affected_groups_level_3', 'specific_needs_groups', 'age', 'gender', 'subpillars_1d', 'subpillars_2d', 'severity']
for col in kept_cols:
    full_df[col] = full_df[col].apply(custom_eval)

/home/selim/anaconda3/envs/deepl/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [30]:
all_cols = ['entry_id', 'excerpt', 'project_id'] + kept_cols 
full_df[full_df.project_id.isin([1621, 2225, 2311])][all_cols].to_csv('summarization_df.csv', index=None)

In [ ]:
full_df['pillars_2d'] = full_df.subpillars_2d.apply(lambda x: [item.split('->')[0] for item in x])
full_df['pillars_1d'] = full_df.subpillars_1d.apply(lambda x: [item.split('->')[0] for item in x])
full_df['affected_groups'] = full_df.affected_groups_level_3.apply(
    lambda x: [item for item in x if item not in ['None', 'Others of Concern']]
)
full_df['sectors'] = full_df.sectors.apply(
    lambda x: [item for item in x if item not in ['Cross']]
)
full_df['n_sectors'] = full_df.sectors.apply(
    lambda x: len(x)
)
full_df['severity_scores'] = full_df['severity'].apply(get_severity_score)

#Counter(flatten(libya_df.severity)).most_common(5)

In [4]:
"""project_id_to_name = {
    1621: 'DFS Libya',
    2225: 'IMMAP/DFS RDC',
    2311: 'IMMAP/DFS Colombia'
}

def preprocess_entry(entry):
    def remove_punct(sentence):
        import string
        return sentence.translate(str.maketrans('', '', string.punctuation))

    def omit_stop_words(sentence):
        return ' '.join([word for word in sentence.split() if word.lower() not in stop_words])

    return omit_stop_words(remove_punct(entry))"""

In [5]:
"""between_sequence_links = [
    'Furthermore, ', 'Aditionally, ', 'Moreover, ', 'Besides, ', 'Aside from that, ', 'Also, ', 'In addition to that, ',
    'On the other hand, ', 'On the other side, '
    ]
final_sent_link = ['Finally, ', 'Ultimately, ']
first_sent_link = ['Firstly, ', 'First of all, ', 'In the first place, ']"""

def get_summary_one_row(row):

    """
    function used for getting summary for one task
    """
    final_summary = []

    original_excerpt = row.excerpt
    severity_scores = row.severity_scores
    top_n_sentences = row.n_clusters

    cosine_similarity_matrix = get_similarity_matrix(original_excerpt)

    graph_one_lang = build_graph(cosine_similarity_matrix)

    scores_pagerank = nx.pagerank(graph_one_lang)
    scores = {key: value * severity_scores[key] for key, value in scores_pagerank.items()}

    top_n_sentence_ids = np.argsort(np.array(list(scores.values())))[::-1][:top_n_sentences]
    used_ids = []

    for id_tmp in top_n_sentence_ids:
        row_id = cosine_similarity_matrix[id_tmp, :]
        top_id_row = np.argsort(row_id)[::-1]
        top_id_row = [id for id in top_id_row if id not in used_ids and id not in top_n_sentence_ids][:1]

        top_2_id_row = [id_tmp] + top_id_row

        used_ids += top_2_id_row

        ranked_sentence = ' '.join([original_excerpt[id_tmp] for id_tmp in (top_2_id_row)]) 
            
        summarized_entries = t2t_generation(ranked_sentence)
        capitalized_summaries = ' '.join([sent.capitalize() for sent in nltk.tokenize.sent_tokenize(summarized_entries)])
        final_summary.append(capitalized_summaries)

    final_summary_str = ' '.join(final_summary)
    
    return final_summary_str

def get_summary_one_part(df, col_name: str):

    preprocessed_df = process_df(df, col_name)

    final_string_one_part = '\n'.join(
        [get_summary_one_row(row) for index, row in preprocessed_df.iterrows()]
    )
    print(final_string_one_part)
    return final_string_one_part

def process_df(df: pd.DataFrame, col_name: str):

    df_copy = df.copy()
    df_copy['tmp_tag_str'] = df_copy[col_name].apply(str)

    grouped_df = df_copy.groupby('tmp_tag_str', as_index=False)[['entry_id', 'excerpt', 'severity_scores']].agg(lambda x: list(x))

    grouped_df['len'] = grouped_df['entry_id'].apply(lambda x: len(x))
    grouped_df = grouped_df[grouped_df.len>=3]
    grouped_df['n_clusters'] = grouped_df['len'].apply(get_number_of_clusters)

    grouped_df.sort_values(by='len', ascending=False, inplace=False).drop(columns='tmp_tag_str', inplace=False)

    return grouped_df

def get_report(full_df, tag: str, project_id: int):
    """
    main function to get full report
    """
    df = full_df[full_df.project_id==project_id].copy()
    df_one_sector = df[df.sectors.apply(lambda x: tag in x)]
    print(df_one_sector.shape)

    summaries = defaultdict()
    """
    # secondary tags
    sec_tags_df = df_one_sector[
        (df_one_sector.affected_groups_level_3.apply(lambda x: len(x)>0)) | 
        (df_one_sector.specific_needs_groups.apply(lambda x: len(x)>0))
        ]
    sec_tags_df['secondary_tags'] = sec_tags_df.apply(
        lambda x: x.specific_needs_groups + x.affected_groups_level_3, axis=1
    )
    summaries['affected_specific'] = get_summary_one_part(sec_tags_df, 'secondary_tags')

    # key trends
    specific_tag_df = df_one_sector[df_one_sector.n_sectors==1]
    specific_tag_df['subpillars'] = specific_tag_df.apply(
        lambda x: x.subpillars_1d + x.subpillars_2d, axis=1
    )
    summaries['key_trends'] = get_summary_one_part(specific_tag_df, 'subpillars')"""


    #relation to other sectors
    many_tags_df = df_one_sector[df_one_sector.n_sectors>1]
    many_tags_df['sectors'] = many_tags_df.sectors.apply(
        lambda x: [item for item in x if item!='Cross']
    )
    relation_to_other_sectors_summary = ''

    many_sectors_df = many_tags_df[many_tags_df.sectors.apply(lambda x: len(x)>2)]
    relation_to_other_sectors_summary += get_summary_one_part(many_sectors_df, 'sectors')

    two_sectors_df = many_tags_df[many_tags_df.sectors.apply(lambda x: len(x)==2)]
    relation_to_other_sectors_summary =\
        relation_to_other_sectors_summary + '\n' + get_summary_one_part(two_sectors_df, 'sectors')

    summaries['other_sectors'] = relation_to_other_sectors_summary

    return summaries

In [6]:
full_report = get_report(full_df, 'Health', 1621)

(377, 27)


In [31]:
"""columns = ['Health']

#for one_project_id in list(project_id_to_name.keys()):
#    print(f'FOR THE PROJECT {project_id_to_name[one_project_id]}')
one_project_id = 1621

for tag in columns:
    print(tag)
    partitions = get_summary(full_df, tag, one_project_id)
    print(partitions)
    print('')"""

"columns = ['Health']\n\n#for one_project_id in list(project_id_to_name.keys()):\n#    print(f'FOR THE PROJECT {project_id_to_name[one_project_id]}')\none_project_id = 1621\n\nfor tag in columns:\n    print(tag)\n    partitions = get_summary(full_df, tag, one_project_id)\n    print(partitions)\n    print('')"